In [ ]:
train_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'
test_dir = '/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

In [ ]:
import os
import random

# **Getting Data Ready**

In [ ]:
import tensorflow as tf

IMG_SIZE = (224,224)
BATCH_SIZE = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 validation_split=0.2,
                                                                 subset='training',
                                                                 seed=42,
                                                                 label_mode='categorical',
                                                                 image_size=IMG_SIZE)
valid_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 validation_split=0.2,
                                                                 subset='validation',
                                                                 seed=42,
                                                                 label_mode='categorical',
                                                                 image_size=IMG_SIZE,
                                                                 shuffle=False)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)

---
# **VISUALIZING DATA**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get the class names from the dataset
class_names = train_data.class_names

# Count the occurrences of each class in the training dataset
train_labels = []
for images, labels in train_data:
    train_labels.extend(np.argmax(labels, axis=1))

class_counts = np.bincount(train_labels)


custom_space=0.5

# Create a bar plot
plt.figure(figsize=(25, 12))
plt.bar(class_names, class_counts, width=1.0 - custom_space)
plt.xlabel('Classes', fontsize=12)
plt.ylabel('Counts', fontsize=12)
plt.title('Class Distribution in Training Dataset')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


In [ ]:
# Get the class names from the dataset
class_names = test_data.class_names

# Count the occurrences of each class in the training dataset
test_labels = []
for images, labels in test_data:
    test_labels.extend(np.argmax(labels, axis=1))

class_counts = np.bincount(test_labels)


custom_space=0.5

# Create a bar plot
plt.figure(figsize=(25, 12))
plt.bar(class_names, class_counts, width=1.0 - custom_space)
plt.xlabel('Classes', fontsize=20)
plt.ylabel('Counts', fontsize=20)
plt.title('Class Distribution in Training Dataset')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


# **Checkpoint**
---


In [ ]:
checkpoint_path = 'fruits_classification_model_checkpoint.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         save_best_only=True)

# **Data Augmentation**


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentation = Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.Rescaling(1/255.)
],name='data_augmenation')

---
# **Model Building**

1. EfficientNetB3
2. MobileNetV2
3. VGG16

**EfficientNetB3**

In [ ]:
base_model_1 = tf.keras.applications.EfficientNetB3(include_top=False)
base_model_1.trainable = False

inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs)
x = base_model_1(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x)
efficient_model = tf.keras.Model(inputs, outputs)

In [ ]:
efficient_model.summary()

In [ ]:
efficient_model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

efficient_model_hist = efficient_model.fit(train_data,
                 epochs=10,
                 validation_data=valid_data,
                 validation_steps=len(valid_data)//BATCH_SIZE,
                 callbacks=[checkpoint_callback])

In [ ]:
efficient_model_loss, efficient_model_acc = efficient_model.evaluate(test_data)

**MobileNetV2**

In [ ]:
base_model_2 = tf.keras.applications.MobileNetV2(include_top=False)
base_model_2.trainable = False

inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs)
x = base_model_2(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x)
mobilenet_model = tf.keras.Model(inputs, outputs)

In [ ]:
mobilenet_model.summary()

In [ ]:
mobilenet_model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


mobilenet_model_hist = mobilenet_model.fit(train_data,
                 epochs=10,
                 validation_data=valid_data,
                 validation_steps= len(valid_data)//BATCH_SIZE,
                 callbacks=[checkpoint_callback])

In [ ]:
mobilenet_model_loss, mobilenet_model_acc = mobilenet_model.evaluate(test_data)

**VGG16**

In [ ]:
base_model_3 = tf.keras.applications.VGG16(include_top=False)
base_model_3.trainable = False

inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs)
x = base_model_3(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x)
vgg16_model = tf.keras.Model(inputs, outputs)

In [ ]:
vgg16_model.summary()

In [ ]:
vgg16_model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


vgg16_model_hist = vgg16_model.fit(train_data,
                 epochs=10,
                 validation_data=valid_data,
                 validation_steps=len(valid_data)//BATCH_SIZE,
                 callbacks=[checkpoint_callback])

In [ ]:
vgg16_model_loss, vgg16_model_acc = vgg16_model.evaluate(test_data)

In [ ]:
print("---------------------------------------------------------------------\n")

print("\t\t Training Loss  |  Training Accuracy ")

print("\nEfficientNetB3 :  {:.4} \t|\t {:.4}%".format(efficient_model_loss, efficient_model_acc*100))
print("\nMobileNetV2    :  {:.4} \t|\t {:.4}%".format(mobilenet_model_loss, mobilenet_model_acc*100))
print("\nVGG16          :  {:.4} \t|\t {:.4}%".format(vgg16_model_loss, vgg16_model_acc*100))

print("\n---------------------------------------------------------------------")